In [ ]:
!pip install -q pypdf

In [ ]:
!pip install -q langchain

In [ ]:
!pip install -q -U langchain-community

In [ ]:
from langchain.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter

In [ ]:
path = input("Enter PDF file path: ")
loader = PyPDFLoader(path)
pages = loader.load()

In [ ]:
splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=20)
docs = splitter.split_documents(pages)

In [ ]:
print(docs)

In [ ]:
!pip install -q chromadb

In [ ]:
!pip install -q sentence-transformers

In [ ]:
!pip install -q --upgrade huggingface_hub

In [ ]:
HUGGINGFACEHUB_API_TOKEN = ""

In [ ]:
import os
os.environ["HUGGINGFACEHUB_API_TOKEN"] = ""

In [ ]:
from langchain.embeddings import HuggingFaceEmbeddings
from langchain_community.vectorstores import Chroma



huggingface_embeddings = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")

vectorstore = Chroma.from_documents(documents=docs, embedding=huggingface_embeddings)

In [ ]:
%pip install -qU langchain-groq

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 108.9/108.9 kB 7.9 MB/s eta 0:00:00


In [ ]:
import getpass
import os

if "GROQ_API_KEY" not in os.environ:
    os.environ["GROQ_API_KEY"] = ""
    # getpass.getpass("Enter your Groq API key:")

In [ ]:
from langchain_groq import ChatGroq

llm = ChatGroq(
    model="llama-3.1-70b-versatile",
    temperature=0.6,
    max_tokens=1024,
    timeout=None,
    max_retries=2,
)

In [ ]:
retriever = vectorstore.as_retriever(search_kwargs={"k": 3})

In [ ]:
from langchain.chains import RetrievalQA

In [ ]:
qa_chain = RetrievalQA.from_chain_type(llm=llm,
                                  chain_type="stuff",
                                  retriever=retriever,
                                  return_source_documents=True)

In [ ]:
query = "que es el pago adicional segun la UCI"
llm_response = qa_chain(query)


In [ ]:
 text = llm_response['result']


In [ ]:
import re

def limpiar_texto(texto):
    texto_sin_caracteres = re.sub(r'[^\w\s]', '', texto)

    parrafos = texto_sin_caracteres.split('\n')

    parrafos = [p for p in parrafos if p.strip()]

    texto_limpio = '\n'.join(parrafos)

    return texto_limpio


texto_limpio = limpiar_texto(text)
print(texto_limpio)